In [1]:
# load package
import numpy as np
import pandas as pd
import jieba
from collections import Counter


# set path
import os
default_path = "/Users/francislin/jrml/What's Cooking/"
os.chdir(default_path)

# skip interaction warning
import warnings; 
warnings.filterwarnings('ignore')
#你可以选择修改 ast_note_iteractively kernel 选项来使得 Jupyter 为每一行的变量或语句执行这个操作，以便你可以立即看到多条语句一起输出
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
train=pd.read_json('./data/train.json')
test=pd.read_json('./data/test.json')
submission=pd.read_csv('./data/sample_submission.csv')

In [3]:
train.head()

,cuisine,id,ingredients
0,greek,10259,"[romaine lettuce, black olives, grape tomatoes..."
1,southern_us,25693,"[plain flour, ground pepper, salt, tomatoes, g..."
2,filipino,20130,"[eggs, pepper, salt, mayonaise, cooking oil, g..."
3,indian,22213,"[water, vegetable oil, wheat, salt]"
4,indian,13162,"[black pepper, shallots, cornflour, cayenne pe..."


In [4]:
test.head()

,id,ingredients
0,18009,"[baking powder, eggs, all-purpose flour, raisi..."
1,28583,"[sugar, egg yolks, corn starch, cream of tarta..."
2,41580,"[sausage links, fennel bulb, fronds, olive oil..."
3,29752,"[meat cuts, file powder, smoked sausage, okra,..."
4,35687,"[ground black pepper, salt, sausage casings, l..."


In [5]:
df=train.append(test, ignore_index=True)
del (train, test)

In [6]:
content = list(df['ingredients'])
contenttxt = ''.join(str(e) for e in content)
#把全部要分析的ingredients灌進一個string

In [7]:
#把結巴沒斷好的拿掉
garbage = ['teeeest']

In [8]:
# 計算詞頻
def get_words(txt, num=900):
    seg_list = jieba.cut(txt, cut_all=False)
    temp = list(seg_list)
    temp = [x.strip() for x in temp]
    temp = [x for x in temp if x not in garbage]
    c = Counter()
    for x in temp:
        if len(x)>1 and x != '\r\n':
            c[x] += 1
    return c.most_common(num)

In [9]:
frq = get_words(contenttxt)
print(frq)
print(type(frq))

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/ps/4m0fjtss0nv0lbhy26n3224m0000gn/T/jieba.cache
Loading model cost 0.824 seconds.
Prefix dict has been built succesfully.


[('pepper', 31909), ('salt', 30573), ('oil', 29048), ('garlic', 23614), ('ground', 22836), ('fresh', 22161), ('sauce', 15987), ('sugar', 15621), ('onions', 15450), ('cheese', 14638), ('chicken', 14300), ('olive', 13539), ('black', 13465), ('water', 12219), ('red', 11525), ('flour', 11082), ('butter', 10804), ('tomatoes', 10717), ('green', 10643), ('powder', 10302), ('chopped', 9802), ('cloves', 9422), ('juice', 8970), ('white', 8626), ('onion', 8609), ('eggs', 8170), ('rice', 7661), ('cream', 7628), ('cilantro', 7432), ('milk', 7353), ('lemon', 7288), ('vegetable', 7068), ('leaves', 6837), ('large', 6787), ('ginger', 6732), ('corn', 6672), ('dried', 6647), ('lime', 6199), ('vinegar', 6185), ('soy', 6076), ('all', 6054), ('purpose', 6045), ('cumin', 5809), ('broth', 5556), ('chili', 5406), ('wine', 5334), ('bell', 5313), ('parsley', 5172), ('sesame', 4391), ('beans', 4353), ('grated', 4093), ('kosher', 4012), ('carrots', 4005), ('extra', 3856), ('basil', 3835), ('beef', 3798), ('dry', 3

In [10]:
#計算出用字詞頻，以利未來製作feature標籤
df_frq = pd.DataFrame(frq, columns=["words","frq"])
df_frq

,words,frq
0,pepper,31909
1,salt,30573
2,oil,29048
3,garlic,23614
4,ground,22836
5,fresh,22161
6,sauce,15987
7,sugar,15621
8,onions,15450
9,cheese,14638


In [11]:
df_frq.to_csv("df_frq.csv", index= False)

In [12]:
def list_2_str(L):
    str1 = ''.join(L)
    return str1

In [13]:
df['ingred']= df['ingredients'].apply(lambda x: list_2_str(x))

In [16]:
#用結巴出來的結果系統性貼標
for a in df_frq['words']:
    df['used_%s'% (a)]= df['ingred'].str.contains('%s'% (a),na=False)*1
    df['used_%s'% (a)].astype('int8')

0        1
1        1
2        1
3        0
4        1
5        0
6        1
7        0
8        1
9        0
10       1
11       1
12       1
13       1
14       1
15       1
16       0
17       0
18       0
19       1
20       0
21       0
22       1
23       1
24       1
25       1
26       1
27       1
28       1
29       0
        ..
49688    1
49689    1
49690    0
49691    1
49692    0
49693    0
49694    1
49695    1
49696    1
49697    1
49698    1
49699    0
49700    1
49701    0
49702    0
49703    1
49704    0
49705    1
49706    0
49707    1
49708    0
49709    0
49710    0
49711    0
49712    1
49713    0
49714    0
49715    1
49716    1
49717    1
Name: used_pepper, Length: 49718, dtype: int8

0        0
1        1
2        1
3        1
4        1
5        1
6        1
7        0
8        1
9        1
10       0
11       0
12       1
13       1
14       1
15       0
16       0
17       0
18       0
19       1
20       0
21       1
22       1
23       0
24       0
25       1
26       0
27       1
28       0
29       0
        ..
49688    0
49689    1
49690    1
49691    0
49692    1
49693    0
49694    1
49695    0
49696    0
49697    0
49698    1
49699    1
49700    1
49701    1
49702    1
49703    1
49704    0
49705    1
49706    1
49707    1
49708    1
49709    0
49710    1
49711    1
49712    1
49713    1
49714    1
49715    1
49716    1
49717    0
Name: used_salt, Length: 49718, dtype: int8

0        0
1        1
2        1
3        1
4        1
5        0
6        1
7        1
8        1
9        1
10       1
11       1
12       1
13       1
14       0
15       0
16       0
17       1
18       0
19       1
20       0
21       0
22       0
23       1
24       1
25       1
26       1
27       0
28       1
29       0
        ..
49688    1
49689    1
49690    0
49691    0
49692    1
49693    0
49694    0
49695    0
49696    1
49697    0
49698    0
49699    0
49700    0
49701    0
49702    1
49703    0
49704    1
49705    1
49706    1
49707    0
49708    0
49709    1
49710    0
49711    1
49712    1
49713    0
49714    0
49715    1
49716    0
49717    1
Name: used_oil, Length: 49718, dtype: int8

0        1
1        0
2        1
3        0
4        1
5        0
6        1
7        0
8        0
9        1
10       1
11       1
12       1
13       1
14       0
15       0
16       0
17       0
18       0
19       1
20       0
21       0
22       0
23       0
24       1
25       0
26       0
27       0
28       1
29       0
        ..
49688    1
49689    0
49690    0
49691    0
49692    0
49693    0
49694    1
49695    0
49696    1
49697    0
49698    0
49699    0
49700    0
49701    0
49702    1
49703    1
49704    0
49705    1
49706    1
49707    1
49708    0
49709    0
49710    0
49711    1
49712    1
49713    0
49714    0
49715    0
49716    0
49717    1
Name: used_garlic, Length: 49718, dtype: int8

0        0
1        1
2        0
3        0
4        1
5        1
6        0
7        0
8        1
9        0
10       0
11       1
12       0
13       1
14       1
15       0
16       0
17       0
18       0
19       0
20       0
21       0
22       0
23       0
24       0
25       0
26       0
27       1
28       1
29       0
        ..
49688    1
49689    1
49690    0
49691    1
49692    0
49693    0
49694    1
49695    0
49696    0
49697    0
49698    0
49699    0
49700    1
49701    0
49702    1
49703    1
49704    0
49705    0
49706    1
49707    0
49708    0
49709    0
49710    0
49711    0
49712    1
49713    0
49714    0
49715    0
49716    0
49717    1
Name: used_ground, Length: 49718, dtype: int8

...

0        0
1        0
2        0
3        0
4        0
5        0
6        0
7        0
8        0
9        0
10       0
11       0
12       0
13       0
14       0
15       0
16       0
17       0
18       0
19       0
20       0
21       0
22       0
23       0
24       0
25       0
26       0
27       0
28       0
29       0
        ..
49688    0
49689    0
49690    0
49691    0
49692    0
49693    0
49694    0
49695    0
49696    0
49697    0
49698    0
49699    0
49700    0
49701    0
49702    0
49703    0
49704    0
49705    0
49706    0
49707    0
49708    0
49709    0
49710    0
49711    0
49712    0
49713    0
49714    0
49715    0
49716    0
49717    0
Name: used_glass, Length: 49718, dtype: int8

0        0
1        0
2        0
3        0
4        0
5        0
6        0
7        0
8        0
9        0
10       0
11       0
12       0
13       0
14       0
15       0
16       0
17       0
18       0
19       0
20       0
21       0
22       0
23       0
24       0
25       0
26       0
27       0
28       0
29       0
        ..
49688    0
49689    0
49690    0
49691    0
49692    0
49693    0
49694    0
49695    0
49696    0
49697    0
49698    0
49699    0
49700    0
49701    0
49702    0
49703    0
49704    0
49705    0
49706    0
49707    0
49708    0
49709    0
49710    0
49711    0
49712    0
49713    0
49714    0
49715    0
49716    0
49717    0
Name: used_safflower, Length: 49718, dtype: int8

0        0
1        0
2        0
3        0
4        0
5        0
6        0
7        0
8        0
9        0
10       0
11       0
12       0
13       0
14       0
15       0
16       0
17       0
18       0
19       0
20       0
21       0
22       0
23       0
24       0
25       0
26       0
27       0
28       0
29       0
        ..
49688    0
49689    0
49690    0
49691    0
49692    0
49693    0
49694    0
49695    0
49696    0
49697    0
49698    0
49699    0
49700    0
49701    0
49702    0
49703    0
49704    0
49705    0
49706    0
49707    0
49708    0
49709    0
49710    0
49711    0
49712    0
49713    0
49714    0
49715    0
49716    0
49717    0
Name: used_jeera, Length: 49718, dtype: int8

0        0
1        0
2        0
3        0
4        0
5        0
6        0
7        0
8        0
9        0
10       0
11       0
12       0
13       0
14       0
15       0
16       0
17       0
18       0
19       0
20       0
21       0
22       0
23       0
24       0
25       0
26       0
27       0
28       0
29       0
        ..
49688    0
49689    0
49690    0
49691    0
49692    0
49693    0
49694    0
49695    0
49696    0
49697    0
49698    0
49699    0
49700    0
49701    0
49702    0
49703    0
49704    0
49705    0
49706    0
49707    0
49708    0
49709    0
49710    0
49711    0
49712    0
49713    0
49714    0
49715    0
49716    0
49717    0
Name: used_Oil, Length: 49718, dtype: int8

0        0
1        0
2        0
3        0
4        0
5        0
6        0
7        0
8        0
9        0
10       0
11       0
12       0
13       0
14       0
15       0
16       0
17       0
18       0
19       0
20       0
21       0
22       0
23       0
24       0
25       0
26       0
27       0
28       0
29       0
        ..
49688    0
49689    0
49690    0
49691    0
49692    0
49693    0
49694    0
49695    0
49696    0
49697    0
49698    0
49699    0
49700    0
49701    0
49702    0
49703    0
49704    0
49705    0
49706    0
49707    0
49708    0
49709    0
49710    0
49711    0
49712    0
49713    0
49714    0
49715    0
49716    0
49717    0
Name: used_oven, Length: 49718, dtype: int8

In [17]:
df.head()

,cuisine,id,ingredients,ingred,used_pepper,used_salt,used_oil,used_garlic,used_ground,used_fresh,...,used_hock,used_rose,used_ciabatta,used_ready,used_regular,used_glass,used_safflower,used_jeera,used_Oil,used_oven
0,greek,10259,"[romaine lettuce, black olives, grape tomatoes...",romaine lettuceblack olivesgrape tomatoesgarli...,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,southern_us,25693,"[plain flour, ground pepper, salt, tomatoes, g...",plain flourground peppersalttomatoesground bla...,1,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,filipino,20130,"[eggs, pepper, salt, mayonaise, cooking oil, g...",eggspeppersaltmayonaisecooking oilgreen chilie...,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,indian,22213,"[water, vegetable oil, wheat, salt]",watervegetable oilwheatsalt,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,indian,13162,"[black pepper, shallots, cornflour, cayenne pe...",black peppershallotscornflourcayenne pepperoni...,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
del (df_frq, frq, contenttxt, content)

In [19]:
df=df.drop(['ingredients','ingred'],axis=1)

In [21]:
import pickle
with open('all_data.pickle','wb') as f:
    pickle.dump(df,f)

In [ ]:
for c in df.columns:
    col_type=df[c].dtype
    if col_type !='object' and c !='id':
        df[c]=df[c].astype('int16')